In [1]:
a = len('DIQMTQSPSSLSASVGDRVTITCRASGNIHNYLAWYQQKPGKAPKLLIYYTTTLADGVPSRFSGSGSGTDYTFTISSLQPEDIATYYCQHFWSTPRTFGQGTKVEIKRQVQLQESGPGLVRPSQTLSLTCTVSGFSLTGYGVNWVRQPPGRGLEWIGMIWGDGNTDYNSALKSRVTMLKDTSKNQFSLRLSSVTAADTAVYYCARERDYRLDYWGQGSLVTVSSKVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRNTDGSTDYGILQINSRWWCNDGRTPGSRNLCNIPCSALLSSDITASVNCAKKIVSDGNGMNAWVAWRNRCKGTDVQAWIRGCRL')
print(a)

353


In [13]:
import pandas as pd
train_df = pd.read_csv('/home/users/hcdai/AI-peptide/Seq2Score/Seq2Score_1.0/dataset/standard_csv_ori/test.csv')
print(train_df.columns)
# ligand_col_name = 'ligand_sequence'
# with open('/home/users/hcdai/AI-peptide/Seq2Score/Seq2Score_1.0/dataset/standard_csv_withpath/test_withpath.csv', 'w') as f:
#             # 写入表头
#     header = ','.join(train_df.columns.tolist() + ['ESMC_Embedding_Path'])
#     f.write(header + '\n')

#             # 逐行处理数据
#     for index, row in train_df.iterrows():
        
#         ligand_seq = row[ligand_col_name]


Index(['ligand_name', 'ligand_sequence', 'receptor_name', 'receptor_sequence',
       'time', 'scores_total_score', 'scores_dslf_fa13', 'scores_fa_atr',
       'scores_fa_dun', 'scores_fa_elec', 'scores_fa_intra_rep',
       'scores_fa_intra_sol_xover4', 'scores_fa_rep', 'scores_fa_sol',
       'scores_hbond_bb_sc', 'scores_hbond_lr_bb', 'scores_hbond_sc',
       'scores_hbond_sr_bb', 'scores_linear_chainbreak', 'scores_lk_ball_wtd',
       'scores_omega', 'scores_overlap_chainbreak', 'scores_p_aa_pp',
       'scores_pro_close', 'scores_rama_prepro', 'scores_ref',
       'scores_yhh_planarity', 'scores_description', 'pack_total_score',
       'pack_complex_normalized', 'pack_dG_cross', 'pack_dG_cross/dSASAx100',
       'pack_dG_separated', 'pack_dG_separated/dSASAx100',
       'pack_dSASA_hphobic', 'pack_dSASA_int', 'pack_dSASA_polar',
       'pack_delta_unsatHbonds', 'pack_dslf_fa13', 'pack_fa_atr',
       'pack_fa_dun', 'pack_fa_elec', 'pack_fa_intra_rep',
       'pack_fa_intra_sol_x

In [ ]:
import torch

# 创建一个示例张量
tensor = torch.randn(64, 65)

# 裁剪前62位
target_1 = tensor[:, :62]

# 裁剪第63位
target_2 = tensor[:, 62:63]

# 裁剪第64位
target_3 = tensor[:, 63:64]

# 裁剪第65位
target_4 = tensor[:, 64:65]

print("target_1 shape:", target_1.shape)
print("target_2 shape:", target_2.shape)
print("target_3 shape:", target_3.shape)
print("target_4 shape:", target_4.shape)

: 

In [ ]:
class S2S_Loss():
    '''
    这个函数应该有的功能：
        应对不同的cut形式
        单独训练和加和训练两种模式
    '''
    def __init__(label_cut
                 train_mode = 'sum'):
        super().__init__()
        self.label_cut = label_cut
        self.train_mode = train_mode
    
    def cut(self,
            model_output:torch_tensor,
            label_cut):
            
    
    def MSE_loss(self):    
    
    def total_loss(self, train_mode):
        if train_mode == 'sum':
            return 1
        if train_mode == 'single':
            return 2

In [ ]:
import torch
import torch.nn as nn

class S2S_Loss():
    '''
    这个函数应该有的功能：
        应对不同的cut形式
        单独训练和加和训练两种模式
    '''
    def __init__(self, 
                 label_cut:list, #[62,1,1,1] # 前62个rosettascore和三个特殊变量
                 train_target:list, # ['RosettaScore','kd','TF','IC50'],大小写区分清楚，只有kd小写
                 adaptive_args:list,# 这玩意必须随着train_target调整啊，是一个必须调整的参数,长度必须为4，因为我实在是懒得接着写if了。eg：[0.3,0.3,0,0.3]
                 ):
        self.label_cut = label_cut
        self.train_target = train_target
        self.adaptive_args = adaptive_args
        self.mse_loss_fn = nn.MSELoss()
        self.bce_loss_fn = nn.BCEWithLogitsLoss()

    def cut(self, model_output: torch.Tensor):
        """
        根据 label_cut 对模型输出进行裁剪
        """
        cut_outputs = []
        start = 0
        for cut in self.label_cut:
            end = start + cut
            cut_outputs.append(model_output[:, start:end])
            start = end
        # cut_output = [[batch_size, 62], [batch_size, 1], [batch_size, 1], [batch_size, 1]] (if label_cut = [62, 1, 1, 1])
        return cut_outputs

    def MSE_loss(self, output, target):
        """
        计算 MSE 损失
        """
        return self.mse_loss_fn(output, target)

    def BCE_loss(self, output, target):
        """
        计算二元交叉熵损失
        """
        return self.bce_loss_fn(output, target)

    def total_loss(self, model_output, targets):
        """
        根据 train_mode 计算总损失
        """
        cut_outputs = self.cut(model_output)
        loss_Rosetta = 0
        loss_TF = 0
        loss_kd = 0
        loss_IC50 = 0
        loss_content = []
        total_loss = 0
        a,b,c,d = self.adaptive_args
        
        for i in self.train_target:
            if i == 'RosettaScore':
                loss_Rossetta = self.MSE_loss(cut_outputs[0], targets[0])
                loss_content.append('RosettaScore')
                continue
            if i == 'TF':
                loss_TF = self.BCE_loss(cut_outputs[1], targets[1])
                loss_content.append('TF')
                continue
            if i == 'kd':
                loss_kd = self.MSE_loss(cut_outputs[2], targets[2])
                loss_content.append('kd')
                continue
            if i == 'IC50':
                loss_IC50 = self.BCE_loss(cut_outputs[3], targets[3])
                loss_content.append('IC50')
                continue
            if i not in ['RosettaScore','TF','kd','IC50']:
                raise ValueError("train_target must be one of ['RosettaScore','TF','kd','IC50']")

        for i in loss_content:
            if i == 'RosettaScore':
                total_loss += a * loss_Rossetta
                continue
            if i == 'TF':
                total_loss += b * loss_TF
                continue
            if i == 'kd':
                total_loss += c * loss_kd
                continue
            if i == 'IC50':
                total_loss += d * loss_IC50
                continue
            if i not in ['RosettaScore','TF','kd','IC50']:
                raise ValueError("train_target must be one of ['RosettaScore','TF','kd','IC50']")
        return total_loss
        

In [ ]:
# 这个版本的S2S_Loss已经可以实现对62个不同回归数值的基于不确定度的回归了，设置optimizer的时候需要注意一下。
# 豆包给的初始化示例代码
# model = ...  # 这里省略模型的定义
# loss_fn = S2S_Loss(label_cut=[62, 1, 1, 1], train_target=['RosettaScore'], adaptive_args=[1.0, 0, 0, 0])
# optimizer = torch.optim.Adam(list(model.parameters()) + [loss_fn.log_var_rosetta], lr=0.001)
# import torch
# import torch.nn as nn

# 训练循环
# for epoch in range(num_epochs):
#     # 前向传播
#     model_output = model(inputs)
#     total_loss = loss_fn.total_loss(model_output, targets)
    
#     # 反向传播
#     optimizer.zero_grad()
#     total_loss.backward()
#     optimizer.step()


class S2S_Loss():
    '''
    这个函数应该有的功能：
        应对不同的cut形式
        单独训练和加和训练两种模式
    '''
    def __init__(self, 
                 label_cut:list, #[62,1,1,1] # 前62个rosettascore和三个特殊变量
                 train_target:list, # ['RosettaScore','kd','TF','IC50'],大小写区分清楚，只有kd小写
                 adaptive_args:list,# 这玩意必须随着train_target调整啊，是一个必须调整的参数,长度必须为4，因为我实在是懒得接着写if了。eg：[0.3,0.3,0,0.3]
                 ):
        self.label_cut = label_cut
        self.train_target = train_target
        self.adaptive_args = adaptive_args
        self.mse_loss_fn = nn.MSELoss()
        self.bce_loss_fn = nn.BCEWithLogitsLoss()
        # 为RosettaScore的62个维度引入独立的不确定性参数
        self.log_var_rosetta = nn.Parameter(torch.zeros(label_cut[0]))

    def cut(self, model_output: torch.Tensor):
        """
        根据 label_cut 对模型输出进行裁剪
        """
        cut_outputs = []
        start = 0
        for cut in self.label_cut:
            end = start + cut
            cut_outputs.append(model_output[:, start:end])
            start = end
        # cut_output = [[batch_size, 62], [batch_size, 1], [batch_size, 1], [batch_size, 1]] (if label_cut = [62, 1, 1, 1])
        return cut_outputs

    def MSE_loss(self, output, target):
        """
        计算 MSE 损失
        """
        return self.mse_loss_fn(output, target)

    def BCE_loss(self, output, target):
        """
        计算二元交叉熵损失
        """
        return self.bce_loss_fn(output, target)

    def uncertainty_mse_loss(self, output, target):
        """
        计算引入不确定性的MSE损失
        """
        precision = torch.exp(-self.log_var_rosetta)
        # 分别计算每个维度的MSE损失
        mse_losses = torch.mean((output - target) ** 2, dim=0)
        # 引入不确定性参数
        losses = precision * mse_losses + self.log_var_rosetta
        # 对所有维度的损失求和
        total_loss = torch.sum(losses)
        return total_loss

    def total_loss(self, model_output, targets):
        """
        根据 train_mode 计算总损失
        """
        cut_outputs = self.cut(model_output)
        loss_Rosetta = 0
        loss_TF = 0
        loss_kd = 0
        loss_IC50 = 0
        loss_content = []
        total_loss = 0
        a,b,c,d = self.adaptive_args
        
        for i in self.train_target:
            if i == 'RosettaScore':
                loss_Rosetta = self.uncertainty_mse_loss(cut_outputs[0], targets[0])
                loss_content.append('RosettaScore')
                continue
            if i == 'TF':
                loss_TF = self.BCE_loss(cut_outputs[1], targets[1])
                loss_content.append('TF')
                continue
            if i == 'kd':
                loss_kd = self.MSE_loss(cut_outputs[2], targets[2])
                loss_content.append('kd')
                continue
            if i == 'IC50':
                loss_IC50 = self.BCE_loss(cut_outputs[3], targets[3])
                loss_content.append('IC50')
                continue
            if i not in ['RosettaScore','TF','kd','IC50']:
                raise ValueError("train_target must be one of ['RosettaScore','TF','kd','IC50']")

        for i in loss_content:
            if i == 'RosettaScore':
                total_loss += a * loss_Rosetta
                continue
            if i == 'TF':
                total_loss += b * loss_TF
                continue
            if i == 'kd':
                total_loss += c * loss_kd
                continue
            if i == 'IC50':
                total_loss += d * loss_IC50
                continue
            if i not in ['RosettaScore','TF','kd','IC50']:
                raise ValueError("train_target must be one of ['RosettaScore','TF','kd','IC50']")
        return total_loss

In [1]:
import torch
import torch.nn as nn

class S2S_Loss():
    '''
    这个函数应该有的功能：
        应对不同的cut形式
        单独训练和加和训练两种模式
    '''
    def __init__(self, 
                 label_cut:list, #[62,1,1,1] # 前62个rosettascore和三个特殊变量
                 train_target:list, # ['RosettaScore','kd','TF','IC50'],大小写区分清楚，只有kd小写
                 adaptive_args:list,# 这玩意必须随着train_target调整啊，是一个必须调整的参数,长度必须为4，因为我实在是懒得接着写if了。eg：[0.3,0.3,0,0.3]
                 alpha: float = 1.5,  # GradNorm 中的比例系数
                 ):
        self.label_cut = label_cut
        self.train_target = train_target
        self.adaptive_args = adaptive_args
        self.mse_loss_fn = nn.MSELoss()
        self.bce_loss_fn = nn.BCEWithLogitsLoss()
        # 为RosettaScore的62个维度引入独立的不确定性参数
        self.log_var_rosetta = nn.Parameter(torch.zeros(label_cut[0]))
        self.alpha = alpha
        # 初始化每个任务的权重，初始值为 1，并设置为可训练参数
        self.weights = nn.Parameter(torch.ones(len(train_target), requires_grad=True))
        # 定义任务优先级
        self.priority = {'IC50': 4, 'kd': 3, 'TF': 2, 'RosettaScore': 1}

    def cut(self, model_output: torch.Tensor):
        """
        根据 label_cut 对模型输出进行裁剪
        """
        cut_outputs = []
        start = 0
        for cut in self.label_cut:
            end = start + cut
            cut_outputs.append(model_output[:, start:end])
            start = end
        # cut_output = [[batch_size, 62], [batch_size, 1], [batch_size, 1], [batch_size, 1]] (if label_cut = [62, 1, 1, 1])
        return cut_outputs

    def MSE_loss(self, output, target):
        """
        计算 MSE 损失
        """
        return self.mse_loss_fn(output, target)

    def BCE_loss(self, output, target):
        """
        计算二元交叉熵损失
        """
        return self.bce_loss_fn(output, target)

    def uncertainty_mse_loss(self, output, target):
        """
        计算引入不确定性的MSE损失
        """
        precision = torch.exp(-self.log_var_rosetta)
        # 分别计算每个维度的MSE损失
        mse_losses = torch.mean((output - target) ** 2, dim=0)
        # 引入不确定性参数
        losses = precision * mse_losses + self.log_var_rosetta
        # 对所有维度的损失求和
        total_loss = torch.sum(losses)
        return total_loss

    def total_loss(self, model_output, targets, model):
        """
        根据 train_mode 计算总损失
        """
        cut_outputs = self.cut(model_output)
        losses = []
        loss_content = []
        task_index = {}

        for i, target_name in enumerate(self.train_target):
            if target_name == 'RosettaScore':
                loss = self.uncertainty_mse_loss(cut_outputs[0], targets[0])
                loss_content.append('RosettaScore')
            elif target_name == 'TF':
                loss = self.BCE_loss(cut_outputs[1], targets[1])
                loss_content.append('TF')
            elif target_name == 'kd':
                loss = self.MSE_loss(cut_outputs[2], targets[2])
                loss_content.append('kd')
            elif target_name == 'IC50':
                loss = self.BCE_loss(cut_outputs[3], targets[3])
                loss_content.append('IC50')
            else:
                raise ValueError("train_target must be one of ['RosettaScore','TF','kd','IC50']")
            losses.append(loss)
            task_index[target_name] = i

        # 计算每个损失项的梯度范数
        grads = []
        for loss in losses:
            # 计算当前损失项对模型参数的梯度
            grad = torch.autograd.grad(loss, model.parameters(), retain_graph=True)
            # 计算梯度的范数
            grad_norm = torch.norm(torch.stack([torch.norm(g) for g in grad]))
            # 将梯度范数添加到列表中
            grads.append(grad_norm)

        # 计算初始梯度范数
        initial_grad_norm = torch.stack(grads).mean()

        # 计算每个损失项的梯度范数相对于初始梯度范数的比例
        relative_grad_norms = [grad / initial_grad_norm for grad in grads]

        # 计算梯度范数的平均值
        avg_relative_grad_norm = sum(relative_grad_norms) / len(relative_grad_norms)

        # 引入任务优先级调整梯度范数差异
        priority_grad_norm_diff = []
        for i, target_name in enumerate(self.train_target):
            priority = self.priority[target_name]
            grad_norm_diff = (relative_grad_norms[i] - avg_relative_grad_norm) ** 2
            priority_grad_norm_diff.append(priority * grad_norm_diff)

        # 计算 GradNorm 损失
        gradnorm_loss = sum(priority_grad_norm_diff) * self.alpha

        # 计算加权后的总损失
        weighted_losses = [w * loss for w, loss in zip(self.weights, losses)]
        total_loss = sum(weighted_losses) + gradnorm_loss

        return total_loss
    
# 示例初始化
label_cut = [62, 1, 1, 1]
train_target = ['RosettaScore', 'kd', 'TF', 'IC50']
adaptive_args = [0.2, 0.2, 0.2, 0.4]
alpha = 1.5

# 创建损失函数实例
loss_fn = S2S_Loss(label_cut, train_target, adaptive_args, alpha)

# 假设的模型输出和目标
model_output = torch.randn(32, sum(label_cut))
targets = [torch.randn(32, 62), torch.randn(32, 1), torch.randn(32, 1), torch.randn(32, 1)]

# 假设的模型
model = nn.Linear(sum(label_cut), sum(label_cut))

# 计算总损失
total_loss = loss_fn.total_loss(model_output, targets, model)
print(total_loss)


KeyboardInterrupt



In [ ]:
# 来个优化文本的版本
import torch
import torch.nn as nn

class S2S_Loss():
    '''
    该类用于计算多任务学习的损失，支持不同的 cut 形式和单独训练、加和训练两种模式。

    参数:
        label_cut (list): 用于对模型输出进行裁剪的列表，例如 [62, 1, 1, 1] 表示前 62 个为 RosettaScore，后三个为特殊变量。
        train_target (list): 训练目标列表，元素必须为 ['RosettaScore', 'kd', 'TF', 'IC50'] 中的一个，大小写区分。
        adaptive_args (list): 自适应参数列表，长度应与 train_target 一致。
        alpha (float): GradNorm 中的比例系数，默认为 1.5。
    '''
    def __init__(self, 
                 label_cut: list,
                 train_target: list,
                 adaptive_args: list,
                 alpha: float = 1.5):
        self.label_cut = label_cut
        self.train_target = train_target
        self.adaptive_args = adaptive_args
        self.mse_loss_fn = nn.MSELoss()
        self.bce_loss_fn = nn.BCEWithLogitsLoss()
        # 为 RosettaScore 的 62 个维度引入独立的不确定性参数
        self.log_var_rosetta = nn.Parameter(torch.zeros(label_cut[0]))
        self.alpha = alpha
        # 初始化每个任务的权重，初始值为 1，并设置为可训练参数
        self.weights = nn.Parameter(torch.ones(len(train_target), requires_grad=True))
        # 定义任务优先级
        self.priority = {'IC50': 4, 'kd': 3, 'TF': 2, 'RosettaScore': 1}

    def cut(self, model_output: torch.Tensor):
        """
        根据 label_cut 对模型输出进行裁剪。

        参数:
            model_output (torch.Tensor): 模型的输出张量。

        返回:
            list: 裁剪后的输出列表，每个元素为一个张量。
        """
        cut_outputs = []
        start = 0
        for cut in self.label_cut:
            end = start + cut
            cut_outputs.append(model_output[:, start:end])
            start = end
        return cut_outputs

    def MSE_loss(self, output, target):
        """
        计算 MSE 损失。

        参数:
            output (torch.Tensor): 模型的输出张量。
            target (torch.Tensor): 目标张量。

        返回:
            torch.Tensor: MSE 损失值。
        """
        return self.mse_loss_fn(output, target)

    def BCE_loss(self, output, target):
        """
        计算二元交叉熵损失。

        参数:
            output (torch.Tensor): 模型的输出张量。
            target (torch.Tensor): 目标张量。

        返回:
            torch.Tensor: 二元交叉熵损失值。
        """
        return self.bce_loss_fn(output, target)

    def uncertainty_mse_loss(self, output, target):
        """
        计算引入不确定性的 MSE 损失。

        参数:
            output (torch.Tensor): 模型的输出张量。
            target (torch.Tensor): 目标张量。

        返回:
            torch.Tensor: 引入不确定性的 MSE 损失值。
        """
        precision = torch.exp(-self.log_var_rosetta)
        # 分别计算每个维度的 MSE 损失
        mse_losses = torch.mean((output - target) ** 2, dim=0)
        # 引入不确定性参数
        losses = precision * mse_losses + self.log_var_rosetta
        # 对所有维度的损失求和
        total_loss = torch.sum(losses)
        return total_loss

    def _get_loss(self, target_name, cut_outputs, targets):
        """
        根据训练目标名称计算相应的损失。

        参数:
            target_name (str): 训练目标名称，必须为 ['RosettaScore', 'kd', 'TF', 'IC50'] 中的一个。
            cut_outputs (list): 裁剪后的输出列表。
            targets (list): 目标列表。

        返回:
            torch.Tensor: 相应的损失值。
        """
        if target_name == 'RosettaScore':
            return self.uncertainty_mse_loss(cut_outputs[0], targets[0])
        elif target_name == 'TF':
            return self.BCE_loss(cut_outputs[1], targets[1])
        elif target_name == 'kd':
            return self.MSE_loss(cut_outputs[2], targets[2])
        elif target_name == 'IC50':
            return self.BCE_loss(cut_outputs[3], targets[3])
        else:
            raise ValueError("train_target must be one of ['RosettaScore', 'TF', 'kd', 'IC50']")

    def total_loss(self, model_output, targets, model):
        """
        根据 train_mode 计算总损失。

        参数:
            model_output (torch.Tensor): 模型的输出张量。
            targets (list): 目标列表。
            model (torch.nn.Module): 模型对象。

        返回:
            torch.Tensor: 总损失值。
        """
        cut_outputs = self.cut(model_output)
        losses = []
        loss_content = []
        task_index = {}

        for i, target_name in enumerate(self.train_target):
            loss = self._get_loss(target_name, cut_outputs, targets)
            loss_content.append(target_name)
            losses.append(loss)
            task_index[target_name] = i

        # 计算每个损失项的梯度范数
        grads = []
        for loss in losses:
            # 计算当前损失项对模型参数的梯度
            grad = torch.autograd.grad(loss, model.parameters(), retain_graph=True)
            # 计算梯度的范数
            grad_norm = torch.norm(torch.stack([torch.norm(g) for g in grad]))
            # 将梯度范数添加到列表中
            grads.append(grad_norm)

        # 计算初始梯度范数
        initial_grad_norm = torch.stack(grads).mean()

        # 计算每个损失项的梯度范数相对于初始梯度范数的比例
        relative_grad_norms = [grad / initial_grad_norm for grad in grads]

        # 计算梯度范数的平均值
        avg_relative_grad_norm = sum(relative_grad_norms) / len(relative_grad_norms)

        # 引入任务优先级调整梯度范数差异
        priority_grad_norm_diff = []
        for i, target_name in enumerate(self.train_target):
            priority = self.priority[target_name]
            grad_norm_diff = (relative_grad_norms[i] - avg_relative_grad_norm) ** 2
            priority_grad_norm_diff.append(priority * grad_norm_diff)

        # 计算 GradNorm 损失
        gradnorm_loss = sum(priority_grad_norm_diff) * self.alpha

        # 计算加权后的总损失
        weighted_losses = [w * loss for w, loss in zip(self.weights, losses)]
        total_loss = sum(weighted_losses) + gradnorm_loss

        return total_loss

In [ ]:
# 这是一个比较正统的gradnormloss的计算类
class GradNormLoss(nn.Module):
    def __init__(self, num_of_task, alpha=1.5):
        super(GradNormLoss, self).__init__()
        self.num_of_task = num_of_task
        self.alpha = alpha
        self.w = nn.Parameter(torch.ones(num_of_task, dtype=torch.float))
        self.l1_loss = nn.L1Loss()
        self.L_0 = None

    # standard forward pass
    def forward(self, L_t: torch.Tensor):
        # initialize the initial loss `Li_0`
        if self.L_0 is None:
            self.L_0 = L_t.detach() # detach
        # compute the weighted loss w_i(t) * L_i(t)
        self.L_t = L_t
        self.wL_t = L_t * self.w
        # the reduced weighted loss
        self.total_loss = self.wL_t.sum()
        return self.total_loss

    # additional forward & backward pass
    def additional_forward_and_backward(self, grad_norm_weights: nn.Module, 
            optimizer: optim.Optimizer):
        # do `optimizer.zero_grad()` outside
        self.total_loss.backward(retain_graph=True)
        # in standard backward pass, `w` does not require grad
        self.w.grad.data = self.w.grad.data * 0.0

        self.GW_t = []
        for i in range(self.num_of_task):
            # get the gradient of this task loss with respect to the shared parameters
            GiW_t = torch.autograd.grad(
                self.L_t[i], 
                grad_norm_weights.parameters(), # 这个参数的功能是指定哪一些层的参数会被计算
                retain_graph=True, 
                create_graph=True
                )
            # compute the norm
            self.GW_t.append(torch.norm(GiW_t[0] * self.w[i]))
        self.GW_t = torch.stack(self.GW_t) # do not detatch
        self.bar_GW_t = self.GW_t.detach().mean()
        self.tilde_L_t = (self.L_t / self.L_0).detach()
        self.r_t = self.tilde_L_t / self.tilde_L_t.mean()
        grad_loss = self.l1_loss(self.GW_t, self.bar_GW_t * (self.r_t ** self.alpha))
        self.w.grad = torch.autograd.grad(grad_loss, self.w)[0]
        optimizer.step()

        self.GW_ti, self.bar_GW_t, self.tilde_L_t, 
            self.r_t, self.L_t, self.wL_t = None, None, None, None, None, None
        # re-norm
        self.w.data = self.w.data / self.w.data.sum() * self.num_of_task

# This is AN interface.
class GradNormModel:
    def get_grad_norm_weights(self) -> nn.Module:
        raise NotImplementedError(
            "Please implement the method `get_grad_norm_weights`")

# # 定义一个简单的多任务模型，包含共享参数模块
class MultiTaskModel(nn.Module):
    def __init__(self):
        super(MultiTaskModel, self).__init__()
        # 定义共享参数模块
        self.shared_layer = nn.Linear(10, 20)
        # 定义任务特定的输出层
        self.task1_output = nn.Linear(20, 1)
        self.task2_output = nn.Linear(20, 1)

    def forward(self, x):
        # 通过共享层
        shared_output = self.shared_layer(x)
        # 分别通过任务特定的输出层
        task1_output = self.task1_output(shared_output)
        task2_output = self.task2_output(shared_output)
        return task1_output, task2_output

    def get_grad_norm_weights(self):
        return self.shared_layer


# 调用的示例代码
# # 超参数设置
# num_epochs = 10
# batch_size = 32
# learning_rate = 0.01
# num_of_task = 2

# # 实例化模型
# model = MultiTaskModel()

# # 实例化 GradNormLoss
# grad_norm_loss = GradNormLoss(num_of_task)

# # 定义优化器
# optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# # 模拟数据集
# num_samples = 1000
# x = torch.randn(num_samples, 10)
# target1 = torch.randn(num_samples, 1)
# target2 = torch.randn(num_samples, 1)
# dataset = torch.utils.data.TensorDataset(x, target1, target2)
# dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

# # 训练循环
# for epoch in range(num_epochs):
#     running_total_loss = 0.0
#     for batch_x, batch_target1, batch_target2 in dataloader:
#         # 前向传播
#         task1_output, task2_output = model(batch_x)

#         # 计算每个任务的损失
#         loss1 = nn.MSELoss()(task1_output, batch_target1)
#         loss2 = nn.MSELoss()(task2_output, batch_target2)
#         L_t = torch.stack([loss1, loss2])

#         # 计算总损失
#         total_loss = grad_norm_loss(L_t)

#         # 清空梯度
#         optimizer.zero_grad()

#         # 额外的前向和反向传播，调整任务损失权重
#         grad_norm_loss.additional_forward_and_backward(
#             grad_norm_weights=model.get_grad_norm_weights(),
#             optimizer=optimizer
#         )

#         # 再次计算总损失相对于模型参数的梯度
#         total_loss.backward()

#         # 更新模型参数
#         optimizer.step()

#         running_total_loss += total_loss.item()

#     # 打印每个 epoch 的平均总损失
#     print(f'Epoch {epoch + 1}/{num_epochs}, Average Total Loss: {running_total_loss / len(dataloader)}')

In [ ]:
# 最终版本，输出的应该是一个包含各个loss的list，后面使用自定义训练器进行训练（由于需要进行gradnorm）
# 豆包给的初始化示例代码
# model = ...  # 这里省略模型的定义
# loss_fn = S2S_Loss(label_cut=[62, 1, 1, 1], train_target=['RosettaScore'], adaptive_args=[1.0, 0, 0, 0])
# optimizer = torch.optim.Adam(list(model.parameters()) + [loss_fn.log_var_rosetta], lr=0.001)
# import torch
# import torch.nn as nn

# 训练循环
# for epoch in range(num_epochs):
#     # 前向传播
#     model_output = model(inputs)
#     total_loss = loss_fn.total_loss(model_output, targets)
    
#     # 反向传播
#     optimizer.zero_grad()
#     total_loss.backward()
#     optimizer.step()


class S2S_Loss():
    '''
    这个函数应该有的功能：
        应对不同的cut形式
        单独训练和加和训练两种模式
    '''
    def __init__(self, 
                 label_cut:list, #[62,1,1,1] # 前62个rosettascore和三个特殊变量
                 train_target:list, # ['RosettaScore','kd','TF','IC50'],大小写区分清楚，只有kd小写
                 ):
        self.label_cut = label_cut
        self.train_target = train_target
        self.mse_loss_fn = nn.MSELoss()
        self.bce_loss_fn = nn.BCEWithLogitsLoss()
        # 为RosettaScore的62个维度引入独立的不确定性参数
        self.log_var_rosetta = nn.Parameter(torch.zeros(label_cut[0]))

    def cut(self, model_output: torch.Tensor):
        """
        根据 label_cut 对模型输出进行裁剪
        """
        cut_outputs = []
        start = 0
        for cut in self.label_cut:
            end = start + cut
            cut_outputs.append(model_output[:, start:end])
            start = end
        # cut_output = [[batch_size, 62], [batch_size, 1], [batch_size, 1], [batch_size, 1]] (if label_cut = [62, 1, 1, 1])
        return cut_outputs

    def MSE_loss(self, output, target):
        """
        计算 MSE 损失
        """
        return self.mse_loss_fn(output, target)

    def BCE_loss(self, output, target):
        """
        计算二元交叉熵损失
        """
        return self.bce_loss_fn(output, target)

    def uncertainty_mse_loss(self, output, target):
        """
        计算引入不确定性的MSE损失
        """
        precision = torch.exp(-self.log_var_rosetta)
        # 分别计算每个维度的MSE损失
        mse_losses = torch.mean((output - target) ** 2, dim=0)
        # 引入不确定性参数
        losses = precision * mse_losses + self.log_var_rosetta
        # 对所有维度的损失求和
        total_loss = torch.sum(losses)
        return total_loss

    def total_loss(self, model_output, targets):
        """
        根据 train_mode 计算总损失
        """
        cut_outputs = self.cut(model_output)
        loss_Rosetta = 0
        loss_TF = 0
        loss_kd = 0
        loss_IC50 = 0
        loss_content = []
        targets = self.cut(targets)
        
        for i in self.train_target:
            if i == 'RosettaScore':
                loss_Rosetta = self.uncertainty_mse_loss(cut_outputs[0], targets[0])
                loss_content.append(loss_Rosetta)
                del loss_Rosetta
                continue
            if i == 'TF':
                loss_TF = self.BCE_loss(cut_outputs[1], targets[1])
                loss_content.append(loss_TF)
                del loss_TF
                continue
            if i == 'kd':
                loss_kd = self.MSE_loss(cut_outputs[2], targets[2])
                loss_content.append(loss_kd)
                del loss_kd
                continue
            if i == 'IC50':
                loss_IC50 = self.BCE_loss(cut_outputs[3], targets[3])
                loss_content.append(loss_IC50)
                del loss_IC50
                continue
            if i not in ['RosettaScore','TF','kd','IC50']:
                raise ValueError("train_target must be one of ['RosettaScore','TF','kd','IC50']")
            
        return loss_content

In [ ]:
training_args = TrainingArguments(
    output_dir='/home/users/hcdai/AI-peptide/Seq2Score/Seq2Rosscore/ESMC-MLP/ESMC-attn/weight_of_attn_20250125/attn_baseline',
    num_train_epochs=100,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='/home/users/hcdai/AI-peptide/Seq2Score/Seq2Rosscore/ESMC-MLP/ESMC-attn/log_of_attn_20250125/attn_baseline',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    fp16=True,  # 启用混合精度训练
)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
# scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

class CustomTrainer(Trainer):
    def create_optimizer_and_scheduler(self, num_training_steps: int):
        self.optimizer = optimizer
        
    def compute_loss(self, model, inputs, return_outputs=False,num_items_in_batch = None):
        # print("Inputs:", inputs['labels'].size)
        labels = inputs.pop("labels").to(device)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        outputs = model(**inputs, labels=labels)
        loss = outputs[0]
        
        return (loss, outputs[1]) if return_outputs else loss

    def on_epoch_end(self):
        # 获取当前 epoch 的验证损失
        eval_metrics = self.evaluate()
        val_loss = eval_metrics["eval_loss"]
        # 根据验证损失调整学习率
        self.lr_scheduler.step(val_loss)

for epoch in range(training_args.num_train_epochs):
    # 重新划分训练集和验证集，验证集比例为 10%
    train_df, valid_df = train_test_split(output_ESMC, test_size=0.1, random_state=epoch)
    
    # 重置索引，保证索引连续，避免出现KeyError
    train_df = train_df.reset_index(drop=True)
    valid_df = valid_df.reset_index(drop=True)

    train_dataset = PeptideDataset(df = train_df,
                                   score_columns = score_columns,
                                   esmc_hidden_column = 'ESMC_Embedding_Path')

    valid_dataset = PeptideDataset(df = valid_df,
                                   score_columns = score_columns,
                                   esmc_hidden_column = 'ESMC_Embedding_Path')

    trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        data_collator=None,
    )

    # 数据加载器
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=training_args.per_device_train_batch_size,
                                                   shuffle=True, pin_memory=True)
    eval_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=training_args.per_device_eval_batch_size,
                                                  shuffle=False, pin_memory=True)

    # 使用加速器包装模型和数据加载器
    model = accelerator.prepare(model)
    train_dataloader, eval_dataloader = accelerator.prepare(train_dataloader, eval_dataloader)

    # 开始训练
    trainer.train()

    # 保存模型
    model_save_path = f'/home/users/hcdai/AI-peptide/Seq2Score/Seq2Rosscore/ESMC-MLP/ESMC-attn/checkpoint_of_attn_20250125/attn_baseline/attn_baseline_checkpoint_epoch_{epoch}.pth'
    torch.save(model.state_dict(), model_save_path)

    # 评估模型
    eval_results = trainer.evaluate()
    print(f"Epoch {epoch + 1} Evaluation results: {eval_results}")

In [ ]:
class CustomTrainer(Trainer):
    def __init__(self, model, args, train_dataset, eval_dataset, data_collator, label_cut, train_target, accelerator):
        super().__init__(model, args, train_dataset, eval_dataset, data_collator)
        self.label_cut = label_cut
        self.train_target = train_target
        self.accelerator = accelerator
        self.s2s_loss = S2S_Loss(label_cut, train_target)
        self.grad_norm_loss = GradNormLoss(num_of_task=len(train_target))

    def create_optimizer_and_scheduler(self, num_training_steps: int):
        # 这里假设 optimizer 已经在外部定义
        self.optimizer = optimizer
        # 可以根据需要在这里定义学习率调度器

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels").to(self.accelerator.device)
        inputs = {k: v.to(self.accelerator.device) for k, v in inputs.items()}
        outputs = model(**inputs)

        # 计算每个任务的损失
        loss_content = self.s2s_loss.total_loss(outputs, labels)
        loss_content = torch.stack(loss_content)

        # 使用 GradNormLoss 计算加权损失
        total_loss = self.grad_norm_loss(loss_content)

        return (total_loss, outputs) if return_outputs else total_loss

    def training_step(self, model, inputs):
        model.train()
        inputs = self._prepare_inputs(inputs)

        # 清零梯度
        self.optimizer.zero_grad()

        # 计算损失
        loss = self.compute_loss(model, inputs)

        # 反向传播
        self.accelerator.backward(loss)

        # 执行 GradNorm 的额外前向和反向传播步骤
        grad_norm_weights = model.get_grad_norm_weights()
        self.grad_norm_loss.additional_forward_and_backward(grad_norm_weights, self.optimizer)

        # 更新模型参数
        self.optimizer.step()

        return loss.detach()


# 以下是使用示例
# 假设 model 已经定义
model = ...
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
accelerator = Accelerator()

# 训练参数
training_args = TrainingArguments(
    output_dir='/home/users/hcdai/AI-peptide/Seq2Score/Seq2Rosscore/ESMC-MLP/ESMC-attn/weight_of_attn_20250125/attn_baseline',
    num_train_epochs=100,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='/home/users/hcdai/AI-peptide/Seq2Score/Seq2Rosscore/ESMC-MLP/ESMC-attn/log_of_attn_20250125/attn_baseline',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    fp16=True,  # 启用混合精度训练
)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# 假设 output_ESMC 和 score_columns 已经定义
output_ESMC = ...
score_columns = ...

# 损失函数参数
label_cut = [62, 1, 1, 1]
train_target = ['RosettaScore', 'kd', 'TF', 'IC50']

for epoch in range(training_args.num_train_epochs):
    # 重新划分训练集和验证集，验证集比例为 10%
    train_df, valid_df = train_test_split(output_ESMC, test_size=0.1, random_state=epoch)
    
    # 重置索引，保证索引连续，避免出现KeyError
    train_df = train_df.reset_index(drop=True)
    valid_df = valid_df.reset_index(drop=True)

    train_dataset = PeptideDataset(df = train_df,
                                   score_columns = score_columns,
                                   esmc_hidden_column = 'ESMC_Embedding_Path')

    valid_dataset = PeptideDataset(df = valid_df,
                                   score_columns = score_columns,
                                   esmc_hidden_column = 'ESMC_Embedding_Path')

    trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        data_collator=None,
        label_cut=label_cut,
        train_target=train_target,
        accelerator=accelerator
    )

    # 数据加载器
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=training_args.per_device_train_batch_size,
                                                   shuffle=True, pin_memory=True)
    eval_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=training_args.per_device_eval_batch_size,
                                                  shuffle=False, pin_memory=True)

    # 使用加速器包装模型和数据加载器
    model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

    # 开始训练
    trainer.train()

    # 保存模型
    model_save_path = f'/home/users/hcdai/AI-peptide/Seq2Score/Seq2Rosscore/ESMC-MLP/ESMC-attn/checkpoint_of_attn_20250125/attn_baseline/attn_baseline_checkpoint_epoch_{epoch}.pth'
    torch.save(model.state_dict(), model_save_path)

    # 评估模型
    eval_results = trainer.evaluate()
    print(f"Epoch {epoch + 1} Evaluation results: {eval_results}")

In [4]:
import os
import torch

def check_pt_files(folder_path):
    # 遍历文件夹中的所有文件
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.pt'):
                # 构建文件的完整路径
                file_path = os.path.join(root, file)
                try:
                    # 加载.pt文件
                    tensor = torch.load(file_path)
                    # 检查张量的形状是否符合要求
                    if tensor.shape != (36, 1, 256, 1152):
                        print(f"文件 {file_path} 的形状不符合 [1, 256, 1152]，实际形状为 {tensor.shape}")
                except Exception as e:
                    print(f"加载文件 {file_path} 时出现错误: {e}")

# 指定要遍历的文件夹路径
folder_path = '/home/users/hcdai/AI-peptide/Seq2Score/Seq2Score_1.0/dataset/ESMC_embedding/TF_ori_withpath/merge'  # 请替换为实际的文件夹路径
check_pt_files(folder_path)

/tmp/ipykernel_984436/780412936.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tensor = torch.load(file_path)


In [2]:
print(len([
        'scores_total_score', 'scores_dslf_fa13',
        'scores_fa_atr', 'scores_fa_dun', 'scores_fa_elec',
        'scores_fa_intra_rep', 'scores_fa_intra_sol_xover4', 'scores_fa_rep',
        'scores_fa_sol', 'scores_hbond_bb_sc', 'scores_hbond_lr_bb',
        'scores_hbond_sc', 'scores_hbond_sr_bb', 
        'scores_lk_ball_wtd', 'scores_omega', 
        'scores_p_aa_pp', 'scores_pro_close', 'scores_rama_prepro',
        'scores_ref',
        'pack_total_score', 'pack_complex_normalized', 'pack_dG_separated',
        'pack_dG_separated/dSASAx100', 'pack_dSASA_hphobic',
        'pack_dSASA_int', 'pack_dSASA_polar', 'pack_delta_unsatHbonds',
        'pack_dslf_fa13', 'pack_fa_atr', 'pack_fa_dun', 'pack_fa_elec',
        'pack_fa_intra_rep', 'pack_fa_intra_sol_xover4', 'pack_fa_rep',
        'pack_fa_sol', 'pack_hbond_E_fraction', 'pack_hbond_bb_sc',
        'pack_hbond_lr_bb', 'pack_hbond_sc', 'pack_hbond_sr_bb',
        'pack_hbonds_int', 'pack_lk_ball_wtd', 'pack_nres_all',
        'pack_nres_int', 'pack_omega', 'pack_p_aa_pp', 'pack_packstat',
        'pack_per_residue_energy_int', 'pack_pro_close', 'pack_rama_prepro',
        'pack_ref', 'pack_sc_value', 'pack_side1_normalized',
        'pack_side1_score', 'pack_side2_normalized', 'pack_side2_score',
        ]))

56


In [ ]:
import torch
import torch.nn as nn
import csv
import pandas as pd

class S2S_Loss():
    '''
    这个函数应该有的功能：
        应对不同的cut形式
        单独训练和加和训练两种模式
    '''
    def __init__(self, 
                 label_cut:list, #[62,1,1,1] # 前62个rosettascore和三个特殊变量
                 train_target:list, # ['RosettaScore','kd','TF','IC50'],大小写区分清楚，只有kd小写
                 ):
        self.label_cut = label_cut
        self.train_target = train_target
        self.mse_loss_fn = nn.MSELoss()
        self.bce_loss_fn = nn.BCEWithLogitsLoss()
        # 为RosettaScore的62个维度引入独立的不确定性参数
        self.log_var_rosetta = nn.Parameter(torch.tensor([-0.5]* 62))
        
        self.csv_file = open('/home/users/hcdai/AI-peptide/Seq2Score/Seq2Score_1.0/debug/20250217_why_so_big/2.csv', 'w', newline='')
        self.csv_writer = csv.writer(self.csv_file)
        self.csv_row = []

    def cut(self, model_output: torch.Tensor):
        """
        根据 label_cut 对模型输出进行裁剪
        """
        cut_outputs = []
        start = 0
        for cut in self.label_cut:
            end = start + cut
            cut_outputs.append(model_output[:, start:end])
            start = end
        # cut_output = [[batch_size, 62], [batch_size, 1], [batch_size, 1], [batch_size, 1]] (if label_cut = [62, 1, 1, 1])
        return cut_outputs

    def MSE_loss(self, output, target):
        """
        计算 MSE 损失
        """
        print(f'target{target},output{output},mse{self.mse_loss_fn(output, target)}')
        return self.mse_loss_fn(output, target)

    def BCE_loss(self, output, target):
        """
        计算二元交叉熵损失
        """
        if output.shape[1] == 2:
            # 这里假设 target 为 0 或 1，取对应位置的概率
            probabilities = output.gather(1, target.long().unsqueeze(1)).squeeze(1)
            # 计算 BCE 损失
            return self.bce_loss_fn(probabilities, target.squeeze(1))
        
        return self.bce_loss_fn(output, target)

    def uncertainty_mse_loss(self, output, target):
        """
        计算引入不确定性的MSE损失
        """
        # log_var_rosetta = self.log_var_rosetta.to(output.device)
        # precision = torch.exp(-log_var_rosetta)
        # # 分别计算每个维度的MSE损失
        # mse_losses = torch.mean((output - target) ** 2, dim=0)
        # # 引入不确定性参数
        # losses = precision * mse_losses + log_var_rosetta
        # # 对所有维度的损失求和
        # total_loss = torch.sum(losses)
        log_var_rosetta = self.log_var_rosetta.to(output.device)
        # 分别计算每个维度的MSE损失
        # self.csv_row = []
        mse_losses = (output - target) ** 2
        
        # 引入不确定性参数
        loss = (mse_losses/(2 * log_var_rosetta.exp())+log_var_rosetta.exp()/2).sum()
        # loss.backward()
        # self.csv_row.extend(mse_losses)
        # self.csv_row.extend(mse_losses/(2 * log_var_rosetta.exp())+log_var_rosetta.exp()/2)
        # print(f'mse_losses:{mse_losses},loss:{loss},loss_func:{mse_losses/(2 * log_var_rosetta.exp())+log_var_rosetta.exp()/2},model_output:{output},target:{target}')
        
        self.csv_row.append(f'mse_losses:{mse_losses},loss:{loss},loss_func:{mse_losses/(2 * log_var_rosetta.exp())+log_var_rosetta.exp()/2},model_output:{output},target:{target}')
        self.csv_writer.writerow(self.csv_row)
        self.csv_row = []
        return loss

    def total_loss(self, model_output, targets):
        """
        根据 train_mode 计算总损失
        """
        cut_outputs = self.cut(model_output)
        loss_Rosetta = 0
        loss_TF = 0
        loss_kd = 0
        loss_IC50 = 0
        loss_content = []
        targets = self.cut(targets)
        
        for i in self.train_target:
            if i == 'RosettaScore':
                loss_Rosetta = self.MSE_loss(cut_outputs[0], targets[0])
                loss_content.append(loss_Rosetta)
                del loss_Rosetta
                continue
            if i == 'TF':
                loss_TF = self.BCE_loss(cut_outputs[1], targets[1])
                loss_content.append(loss_TF)
                del loss_TF
                continue
            if i == 'kd':
                loss_kd = self.MSE_loss(cut_outputs[2], targets[2])
                loss_content.append(loss_kd)
                del loss_kd
                continue
            if i == 'IC50':
                loss_IC50 = self.BCE_loss(cut_outputs[3], targets[3])
                loss_content.append(loss_IC50)
                del loss_IC50
                continue
            if i not in ['RosettaScore','TF','kd','IC50']:
                raise ValueError("train_target must be one of ['RosettaScore','TF','kd','IC50']")
        
        return loss_content
    
    database_path = '/home/users/hcdai/AI-peptide/Seq2Score/Seq2Score_1.0/trying/20250217_loss_too_big/dataset/generated_EK_withpath_train_try_deleted.csv'
    label_cut = [56]
    train_target = ['RosettaScore']
    score_columns = [
        'scores_total_score', 'scores_dslf_fa13',
        'scores_fa_atr', 'scores_fa_dun', 'scores_fa_elec',
        'scores_fa_intra_rep', 'scores_fa_intra_sol_xover4', 'scores_fa_rep',
        'scores_fa_sol', 'scores_hbond_bb_sc', 'scores_hbond_lr_bb',
        'scores_hbond_sc', 'scores_hbond_sr_bb', 
        'scores_lk_ball_wtd', 'scores_omega', 
        'scores_p_aa_pp', 'scores_pro_close', 'scores_rama_prepro',
        'scores_ref',
        'pack_total_score', 'pack_complex_normalized', 'pack_dG_separated',
        'pack_dG_separated/dSASAx100', 'pack_dSASA_hphobic',
        'pack_dSASA_int', 'pack_dSASA_polar', 'pack_delta_unsatHbonds',
        'pack_dslf_fa13', 'pack_fa_atr', 'pack_fa_dun', 'pack_fa_elec',
        'pack_fa_intra_rep', 'pack_fa_intra_sol_xover4', 'pack_fa_rep',
        'pack_fa_sol', 'pack_hbond_E_fraction', 'pack_hbond_bb_sc',
        'pack_hbond_lr_bb', 'pack_hbond_sc', 'pack_hbond_sr_bb',
        'pack_hbonds_int', 'pack_lk_ball_wtd', 'pack_nres_all',
        'pack_nres_int', 'pack_omega', 'pack_p_aa_pp', 'pack_packstat',
        'pack_per_residue_energy_int', 'pack_pro_close', 'pack_rama_prepro',
        'pack_ref', 'pack_sc_value', 'pack_side1_normalized',
        'pack_side1_score', 'pack_side2_normalized', 'pack_side2_score',
        ]
    s2s_loss = S2S_Loss(label_cut, train_target)
    database = pd.read_csv(database_path)
    
    train_df, valid_df = train_test_split(database, test_size=0.1, random_state=42)
    train_df = train_df.reset_index(drop=True)
    valid_df = valid_df.reset_index(drop=True)
    
    scores = train_df.loc[1, score_columns].values
    scores = scores.astype(float)
    scores = torch.tensor(scores, dtype=torch.float).to('cpu')
    
    print(scores)

: 

In [1]:
from esm.models.esmc import ESMC
from esm.sdk.api import ESMProtein, LogitsConfig
import torch
import torch.nn as nn
import pandas as pd
import torch
from transformers import EsmTokenizer, EsmModel, Trainer, TrainingArguments
from accelerate import Accelerator
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn.utils import clip_grad_norm_
from sklearn.preprocessing import StandardScaler